# 2018-04-19 v7 genome gene expression


## Aim

Gene expression table can help prioritize candidate gene by selecting only genes expressed in the stage of interest. To generate such expression table, we need RNA-seq data. Several dataset are already publish for a lot of stage. This is the list of dataset I will use:
* Eggs from [Anderson *et al.*, 2015](https://doi.org/10.1371/journal.pntd.0004334)
* Miracidia from [Wang *et al.*, 2013](https://doi.org/10.7554/eLife.00768.001)
* Cercariae, schistosomula 3h and 24h (*in vitro* transformation) from [Protasio *et al.*, 2012](https://doi.org/10.1371/journal.pntd.0001455)
* Sporocyst 48h (*in vitro* transformation) from [Wang *et al.*, 2013](https://doi.org/10.7554/eLife.00768.001)
* Juveniles (single sex) (18, 21, 28 day old) from [Protasio *et al.*, 2017](https://doi.org/10.1371/journal.pntd.0005559)
* Adults (separate males and females from mix infections) [Protasio *et al.*, 2017](https://doi.org/10.1371/journal.pntd.0005559)

Data will be aligned against the v7 genome of *S. mansoni*.


## Tool preparation

STAR will be used to align data and RSEM will be used to generate transcript per million (TPM) counts. These two tools require a step to prepare the reference genome.

### STAR reference genome

Creating a STAR reference genome requires the use of an annotation file. The Sanger Institute provided us with a GFF file which a format that can be normally used with STAR. However my first attempt to generate a STAR reference genome using the `--sjdbGTFtagExonParentTranscript Parent` option as mentioned in the manual did not allow me to get gene counts after running STAR on sample (the gene count file contains only the first 4 lines). This problem is very similar to [this](https://groups.google.com/forum/#!msg/rna-star/oRvzihFXE8k/Xa-7YgUUBgAJ). Therefore I converted the GFF file into a GTF file which is the default format used by STAR and this solved the problem.

Because the data used were generated on different platforms that produced different read sizes, I made two reference genomes using different values for the `--sjdbOverhang` option as recommended in the STAR documentation:
* A value of 75 for libraries that have 76 bp paired-end reads (the Protasio *et al* 2012).
* A value of 99 for libraries that have 100 bp paired-end reads (all the others).

In [ ]:
# Generate GTF file
cd ~/data/sm_Gene_table/
gffread Sm_v7.1_renamed.gff -T -o Sm_v7.1_renamed.gtf

# Generate ref genome for STAR for 76 and 100 bp paired-end reads as well as pyrosequencing reads
## Max read length from pyrosequencing data was obtained after downloading the library.
## Here was the command line used (source: https://www.biostars.org/p/304055/#304062):
## gunzip -c data/egg/egg_R2.fastq.gz | paste - - - - | awk -F '\t' '{L=length($2);if(L>M) {M=L;R=$0;}} END {print R;}' | tr "\t" "\n" | h
cd ~/data/sm_genome/

length="75 99 2043"

for i in $length
do
    # Make ref folder
    mkdir Smansoni_v7_renamed_STAR_${i}
    cd Smansoni_v7_renamed_STAR_${i}

    STAR --runMode genomeGenerate \
         --runThreadN $(nproc)    \
         --genomeDir ~/data/sm_genome/Smansoni_v7_renamed_STAR_${i} \
         --genomeFastaFiles ~/data/sm_genome/Smansoni_v7_renamed.fa \
         --sjdbGTFfile ~/data/sm_Gene_table/Sm_v7.1_renamed.gtf     \
         --sjdbOverhang $i
         
    cd ..
done


### RSEM reference

RSEM requires to generate a reference using the GFF and the reference genome file as mentioned in the documentation.

In [ ]:
mydir="~/data/sm_genome/Smansoni_v7_renamed_RSEM"
mkdir $mydir
cd $mydir

rsem-prepare-reference --gff3 ../../sm_Gene_table/Sm_v7.1_renamed.gff ../Smansoni_v7_renamed.fa Smansoni_v7_renamed > log

## Data download

In [ ]:
# Project directory
mydir="$HOME/analyses/00 - Misc/2018-04-19 v7 genome gene expression"
mkdir -p "$mydir"
cd "$mydir"

# Data dir
mkdir data

### Protasio *et al.*, 2012

Data are available from EBI (accession number [E-MTAB-451](https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-451/)).

In [ ]:
# Download the metadata file
wget "https://www.ebi.ac.uk/arrayexpress/files/E-MTAB-451/E-MTAB-451.sdrf.txt"

# EBI Runs
myspls=$(cut -f 1 E-MTAB-451.sdrf.txt | tail -n +2 | sort | uniq)

for s in $myspls
do
    mylines=$(awk -F "\t" -v r=$s '$1 == r {print $1"\t"$32}' E-MTAB-451.sdrf.txt)

    echo "$mylines" | while read line
    do
        myfile=$(echo "$line" cut -f 2 | cut -d "/" -f 8 | cut -d "." -f 1)
        
        [[ ! -d "data/$s" ]] && mkdir "data/$s"
        wget -P data/$s/ $(echo "$line" | cut -f 2)
    done
    
    if [[ $(echo "$mylines" | wc -l) -le 2 ]]
    then
        mv data/$s/*_1.fastq.gz data/$s/${s}_R1.fastq.gz
        mv data/$s/*_2.fastq.gz data/$s/${s}_R2.fastq.gz
    else
        cat data/$s/*_1.fastq.gz > data/$s/${s}_R1.fastq.gz && rm data/$s/*_1.fastq.gz
        cat data/$s/*_2.fastq.gz > data/$s/${s}_R2.fastq.gz && rm data/$s/*_2.fastq.gz
    fi
done

### Protasio *et al.*, 2017

Data are available on EBI (accession number [E-ERAD-478](https://www.ebi.ac.uk/arrayexpress/experiments/E-ERAD-478/)) but the metadata were not informative enough to know what libraries correspond to a given stage. So I ask directly Anna Protasio who gave me the information (output.csv file). Below the explanation she emailed me:

>With Nancy's help, I managed to map the accession numbers to the metadata that we have. Rest assure that we will update these samples in ArrayExpress as soon as we can. 
>
>You will find that all the sequencing files are repeated (NNNNN_3_X). N corresponds to the Illumina sequencing run. Originally, these technology (HiSeq2500 I think) splits each library into two lanes, NNNN_1 and NNNNN_2 but they are the same sample. I tend to merge them into one, hence the _3,  but the submission was still done at the individual (_1 and _2) level. Basically, if you want to analyse these data you can merge both experiments with the same "Item" identification. 
>
>One more thing, I seem to be missing  metadata for some of the samples found in the ArrayExpress submission. I reckon these were not used in the analyses and therefore I have no information of what they are at hand. We will dig this up and place it in ArrayExpresss as part of the metadata improvement. In the meantime, I believe all the data that was used (male, female, D18 to D35 post infection, Paired and Unpaired aka virgins) is there. 

Only paired worms from immature (D21 and 28) and adult stage (D38) will be used.

In [ ]:
# Download the metadata file
wget "https://www.ebi.ac.uk/arrayexpress/files/E-ERAD-478/E-ERAD-478.sdrf.txt"

## Load the output.csv file ##

# Number of pair and unpaired worm libraries
for i in $(cut -d "," -f 6 output.csv | sort | uniq)
do 
    mylines=$(awk -F "," -v i=$i '$6 == i {print $9}' output.csv)
    echo "$i $(echo "$mylines" | grep -c U) $(echo "$mylines" | grep -c P)"
done

# EBI Runs
mystgs="D21 D28 D38"

for s in $mystgs
do
    
    mylines=$(awk -F "," -v r=$s '$6 == r && $9 == "P" {print $0}' output.csv)

    echo "$mylines" | while read line
    do
        id=$(echo "$line" | cut -d "," -f 7)
        myrun=$(echo "$line" | cut -d "," -f 3)
        
        data_dir="data/worms_${s}_${id}"
        
        echo "$data_dir"
        
        [[ ! -d "$data_dir" ]] && mkdir "$data_dir"
        fastq-dump -O "$data_dir" --split-files --gzip -A $myrun
        
    done
done


# Rename fastq
for s in $mystgs
do
    # List directories
    mydir=$(find data -maxdepth 1 | grep $s)
    
    for d in $mydir
    do
        myname=${d#*/}
        if [[ $(ls -1 "$d"/*.fastq.gz | wc -l) -le 2 ]]
        then
            mv $d/*_1.fastq.gz $d/${myname}_R1.fastq.gz
            mv $d/*_2.fastq.gz $d/${myname}_R2.fastq.gz
        else
            cat $d/*_1.fastq.gz > $d/${myname}_R1.fastq.gz && rm $d/*_1.fastq.gz
            cat $d/*_2.fastq.gz > $d/${myname}_R2.fastq.gz && rm $d/*_2.fastq.gz
        fi
    done
done

### Wang *et al.*, 2013

Data are available on [NCBI](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE48282).

In [ ]:
## Run table downloaded from https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRP026308 and renamed SraRunTable.Miracidium-Sp.txt

while read line
do
    spl=$(echo "$line" | cut -f 8)
    acc=$(echo "$line" | cut -f 5)
    
    [[ $spl =~ "sporocyst" ]] && spl="sporocyst_48h"
    
    # Create sample folder
    [[ ! -d "data/$spl" ]] && mkdir "data/$spl"
    
    # Download from NCBI
    fastq-dump -O "data/$spl" --split-files --gzip -A $acc
    
    # Rename files
    mv "data/$spl/"*_1.fastq.gz "data/$spl/${spl}_R1.fastq.gz"
    
done < <(tail -n +2 SraRunTable.Miracidium-Sp.txt)

### Anderson *et al.*, 2015

Data are available on [NCBI](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA294789).

In [ ]:
## Run table downloaded from https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRP063353 and renamed SraRunTable-Anderson2015.txt

while read line
do
    spl=$(echo "$line" | cut -f 9)
    acc=$(echo "$line" | cut -f 7)
    
    [[ $spl =~ "egg" ]] && spl="egg" || continue
    
    # Create sample folder
    [[ ! -d "data/$spl" ]] && mkdir "data/$spl"
    
    # Download from NCBI
    fastq-dump -O "data/$spl" --split-files --gzip -A $acc
    
    # Rename files
    mv "data/$spl/"*_1.fastq.gz "data/$spl/${spl}_R1.fastq.gz"
    mv "data/$spl/"*_2.fastq.gz "data/$spl/${spl}_R2.fastq.gz"
    
done < <(tail -n +2 SraRunTable.Anderson2015.txt)

## Processing samples

Now that fastq files were downloaded, data will be aligned using STAR and the appropriate reference genome depending on the read length and transcript per million (TPM) will be generated using RSEM.

**Note 1**: Egg data has been excluded because generating an error during the alignment steps (`EXITING because of FATAL ERROR in reads input: quality string length is not equal to sequence length`) produced on read 13. Using Trimomatic may solve the problem but was not tested (is Trimomatic good for 454 data?). If e

**Note 2**: I had trouble on medusa2 server to run RSEM on nodes because of perl library matching problem. Therefore I run the loop on titan server.

In [ ]:
# List samples
#myspl=("egg" \
myspl=("miracidia"     \
       "sporocyst_48h" \
       "cerc10a cerc12 cerc13" \
       "somule1 somule2"       \
       "somule3 somule4 somule5 somule6" \
       "worms_D21_P1F worms_D21_P2F worms_D21_P3F worms_D21_P5F" \
       "worms_D21_P1M worms_D21_P2M worms_D21_P3M worms_D21_P5M" \
       "worms_D28_P1F worms_D28_P3F worms_D28_P4F worms_D28_P5F" \
       "worms_D28_P1M worms_D28_P3M worms_D28_P4M worms_D28_P5M" \
       "worms_D38_P1F worms_D38_P2F worms_D38_P45F" \
       "worms_D38_P1M worms_D38_P2M worms_D38_P45M")

# List sample types
#mytyp=(egg \
mytyp=(miracidia   \
       sporocyst_48h \
       cercariae   \
       somule_3h   \
       somule_24h  \
       adult_m_21d \
       adult_f_21d \
       adult_m_28d \
       adult_f_28d \
       adult_m_38d \
       adult_f_38d)

# List library read length
#myrl=(657 100 100 76 76 76 100 100 100 100 100 100)
myrl=(100 100 76 76 76 100 100 100 100 100 100)

# List paired-end status
#myps=(single single single paired paired paired paired paired paired paired paired paired)
myps=(single single paired paired paired paired paired paired paired paired paired)


# Prepare folders and links
for ((i=0 ; i<${#mytyp[@]} ; i++))
do
    mkdir -p alignment/${mytyp[i]}

    
    [[ $(echo ${myspl[i]} | wc -w) > 1 ]] && myfd="{$(echo ${myspl[i]} | sed s"/ /,/g")}" || myfd=${myspl[i]}
    
    eval ln -s $(printf '%q' "$PWD")/data/$myfd/*fastq.gz $(printf '%q' "$PWD")/alignment/${mytyp[i]}/

done

## Remove one link manually because the reads seems to correspond to tags
#rm alignment/egg/egg_R1.fastq.gz


# Run STAR on samples
ssh medusa2

[[ ! -d status ]] && mkdir status

# Run the alignment for each sample
for ((i=0 ; i<${#mytyp[@]} ; i++))
do
    tag=$(( ${myrl[i]} -1 ))
    d=alignment/${mytyp[i]}
    qsub -V -cwd -o status/ -j y -r y -b yes   \
        ~/local/bin/STAR --runMode alignReads  \
                         --runThreadN $(nproc) \
                         --genomeDir ~/data/sm_genome/Smansoni_v7_renamed_STAR_$tag \
                         --readFilesIn $(ls -1 $d/*_{R,}1.fastq.gz 2> /dev/null | tr "\n" "," | sed "s/,$//") $(ls -1 $d/*_{R,}2.fastq.gz 2> /dev/null | tr "\n" "," | sed "s/,$//") \
                         --readFilesCommand zcat             \
                         --outFileNamePrefix $d/${d##*/}_    \
                         --outSAMtype BAM SortedByCoordinate \
                         --quantMode TranscriptomeSAM GeneCounts
done



## Wait until jobs done ##

# Run RSEM for quantification
for ((i=0 ; i<${#mytyp[@]} ; i++))
do
    # Working folder 
    d=alignment/${mytyp[i]}
    
    # Library type for activating RSEM option
    [[ ${myps[i]} == single ]] && lib_type="" || lib_type="--paired-end"
    
    # RSEM process
    qsub -V -cwd -o status/ -j y -r y -b yes \
        ~/local/bin/rsem-calculate-expression --alignments $lib_type -p $(nproc) --no-bam-output\
                $d/${d##*/}_Aligned.toTranscriptome.out.bam \
                ~/data/sm_genome/Smansoni_v7_renamed_RSEM/Smansoni_v7_renamed \
                $d/${d##*/}_rsem
done

## Wait until jobs done ##

Once the jobs done, isoform TPM files are availble for each stage of interest. The final step consists in concatenating this data in a single table.

In [ ]:
# Generate table
final_tb=TPM_isoforms_Sm_v7.1.tsv
cut -f 1,6 alignment/${mytyp[0]}/${mytyp[0]}_rsem.isoforms.results > "$final_tb"

for ((i=1 ; i<${#mytyp[@]} ; i++))
do
    join "$final_tb" <(cut -f 1,6 alignment/${mytyp[i]}/${mytyp[i]}_rsem.isoforms.results) > "${final_tb}.tmp"
    mv "${final_tb}.tmp" "${final_tb}"
done

sed -i "1 s/ .*/ $(echo ${mytyp[@]})/g ; s/ /\t/g" "${final_tb}"


# Link the table to the global data directory
ln -s "$PWD/${final_tb}" ~/data/sm_Gene_table/